In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
import pickle as pkl

In [42]:
data=pd.read_csv("Churn_Modelling.csv")

In [43]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [44]:
##preprocessing
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [45]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [46]:
ohe_geography = OneHotEncoder()
geography_encoder = ohe_geography.fit_transform(data[['Geography']]).toarray()
geography_encoder_df = pd.DataFrame(geography_encoder, columns=ohe_geography.get_feature_names_out(['Geography']))
geography_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [47]:
data=pd.concat([data.drop('Geography',axis=1),geography_encoder_df], axis=1)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
##divide the dataset into dependent and independent features
x=data.drop('EstimatedSalary',axis=1) 
y=data['EstimatedSalary']  

##split the dataset into training and testing sets
X_train, X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

##scale the features
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)



In [49]:
with open('label_encoder_gender.pkl', 'wb') as f:
    pkl.dump(label_encoder_gender,f)

with open('ohe_geography.pkl', 'wb') as f:
    pkl.dump(ohe_geography,f)

with open('scaler.pkl', 'wb') as f:
    pkl.dump(scaler,f)


In [50]:
## ann regression problem statement

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [51]:
##build the model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)##output layer for the regression
])


##compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

c:\Users\karti\OneDrive\Desktop\ann classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [53]:
##set up the early stopping
earlystopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [54]:
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[earlystopping_callback,tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100378.9141 - mae: 100378.9141 - val_loss: 98517.2578 - val_mae: 98517.2578
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99615.1406 - mae: 99615.1406 - val_loss: 96968.6953 - val_mae: 96968.6953
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 96906.9766 - mae: 96906.9766 - val_loss: 92992.0547 - val_mae: 92992.0547
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 91584.9375 - mae: 91584.9375 - val_loss: 86384.6250 - val_mae: 86384.6250
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 83829.6172 - mae: 83829.6172 - val_loss: 77824.8281 - val_mae: 77824.8281
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 74686.0234 - mae: 74686.0234 - val_loss: 68922.5703 - val_mae: 68922.5703
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 65885.0234 - mae: 65885.0234 - val_loss: 61234.2266 - val_mae: 61234.2266
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 

In [55]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [56]:

%tensorboard --logdir regressionlogs/fit 

Reusing TensorBoard on port 6006 (pid 25672), started 0:23:18 ago. (Use '!kill 25672' to kill it.)

In [57]:
##evalute model on the test data
train_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Train Loss: {train_loss:.3f}, Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50188.4609 - mae: 50188.4609
Train Loss: 50188.461, Test MAE: 50188.4609375


In [58]:
model.save('regression_model.keras')